<a href="https://colab.research.google.com/github/AUNitman/test_ml_classifier/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# to_csv('/content/drive/MyDrive/Проекты/Titanic_kaggle/clean_train.csv', index=False)

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

[]


### Load datasets

In [4]:
img_height, img_width = 150, 150
batch_size = 32
num_classes = 3

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Проекты/Classifier of X-ray images/train',  # путь к директории с тренировочными данными (папки: frontal, lateral, trash)
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Так как у нас три класса, используем 'categorical'
)

Found 4264 images belonging to 3 classes.


In [9]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/Проекты/Classifier of X-ray images/validation',  # путь к директории с валидационными данными
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 999 images belonging to 3 classes.


### Model

In [15]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Для предотвращения переобучения
    Dense(num_classes, activation='softmax')  # Финальный слой для 3 классов с активацией softmax
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train model


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=20  # Выберите оптимальное количество эпох
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.6041 - loss: 0.9449

In [ ]:
# Оценка модели
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy * 100:.2f}%')